> Copyright 2022 University of Luxembourg
> 
> Licensed under the Apache License, Version 2.0 (the "License");  
> you may not use this file except in compliance with the License.  
> You may obtain a copy of the License at  
>
>    https://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software  
> distributed under the License is distributed on an "AS IS" BASIS,  
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
> See the License for the specific language governing permissions and  
> limitations under the License.  
>
***

Author: André Stemper (andre.stemper@uni.lu)

***

In [ ]:
if not 'experiment_timestamp_string' in locals():
    experiment_timestamp_string='20221021-095246'

if not 'measurements_root' in locals():
    measurements_root='./measurements' 

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import time
import pickle
import sys
import json

enable_example = False
%run dataset.ipynb


Measurement results directory

In [ ]:
measurements_directory = measurements_root + os.path.sep + experiment_timestamp_string

**Load settings**

In [ ]:
filename = measurements_directory + os.path.sep + 'settings.json'
with open(filename, 'r') as file:
    settings = json.load(file)
    print("Settings loaded from '{}'".format(filename))

print('Settings:')
for k, v in settings.items():
    print(" - {} = {}".format(k, v))

**Features**  
Features to be packed into a datapoint and send to the implementaion. The order of the features must correspond to the ordering of a datapoint in the C++ implementation


In [ ]:
feature_columns = ['temp_0', 'temp_1', 'temp_2', 'temp_3',
                   'temp_4', 'temp_5', 'temp_6', 'temp_7', 'temp_8']

In [ ]:
dataset = Dataset(settings['dataset'])[settings['range']]


In [ ]:
fig, ax = plt.subplots(1, 1, sharex=True)
fig.set_figwidth(20)
fig.set_figheight(8)
dataset.plot(columns=feature_columns, ax=ax).plot_anomalies(ax=ax)


**Load the specified dataset**


In [ ]:


# load the dataframe extended with measurement data 
filename = measurements_directory + os.path.sep + "testbench_data.csv"
dataset.overwrite_dataframe_from_csv(filename=filename)


**Load statistics** 

In [ ]:
filename = measurements_directory + os.path.sep + "execution_time_statistics.json"
with open(filename, 'r') as file:
    execution_time_statistics = json.load(file)  
    print("Statistics loaded from '{}'".format(filename))

print("Execution time / datapoint statistics")
for k, v in execution_time_statistics.items():
    print("{} \t= {} [s]".format(k, v))

### Postprocessing 
**Visualize the dataset and the results**


In [ ]:

ylabel_position = (-0.030, 0.5)
fig, axs = plt.subplots(4, 1, sharex=True)
fig.set_figwidth(26)
fig.set_figheight(15)

axs[0].set_title("Dataset {} evaluated on {}@{}MHz, ({})".format(settings['dataset'], settings['connection_type'], settings['frequency'], experiment_timestamp_string),fontsize=16)
axs[3].set_ylabel("Temperature [°C]")
axs[3].yaxis.set_label_coords(*ylabel_position)
dataset.plot(columns=feature_columns, ax=axs[3]).plot_anomalies(ax=axs[3])

axs[0].set_ylabel("Mahalanobis distance squared")
axs[0].yaxis.set_label_coords(*ylabel_position)
dataset.plot(columns=['distance'], ax=axs[0], grid=True).plot_anomalies(ax=axs[0])
axs[0].grid(axis='x')


axs[1].set_ylabel("Detection")
axs[1].yaxis.set_label_coords(*ylabel_position)
axs[1].set_yticks([1.0, 0.0], ["abnormal", "normal"])
dataset.plot(columns=['detection'], ax=axs[1], color='C4').plot_anomalies(ax=axs[1])


axs[2].set_ylabel("Execution time [s]")
axs[2].yaxis.set_label_coords(*ylabel_position)
dataset.plot(columns=['execution_time'], ax=axs[2], grid=True, color='g').plot_anomalies(ax=axs[2])
axs[2].grid(axis='x')

axs[3].set_xlabel("Timestamp [Month-Day Hour]")


fig.subplots_adjust(hspace=0.05)
filename = measurements_directory + os.path.sep + "postprocessing_testbench_results.png"
plt.savefig(filename)
filename = measurements_directory + os.path.sep + "postprocessing_testbench_results.pdf"
fs = plt.rcParams.get('font.size')
plt.rcParams.update({'font.size': 20})
plt.savefig(filename, format='pdf', bbox_inches='tight', pad_inches=0.1)
plt.rcParams.update({'font.size': fs})

**Histogram of excution duration**

In [ ]:
column = dataset.dataframe['execution_time']

if settings['connection_type'] == "local":
    ax=column.plot.hist(column=["execution_time"], bins=15, logy=True, xlim=[0,0.001], figsize=(15,8), edgecolor='white', linewidth=3, grid=True)
else:
    ax=column.plot.hist(column=["execution_time"], bins=15, logy=True, xlim=[0,0.5], figsize=(15,8), edgecolor='white', linewidth=3, grid=True)

ax.set_xlabel("Execution time [s]")
ax.set_ylabel("Frequency of occurrences [1]")

filename = measurements_directory + os.path.sep + "postprocessing_testbench_histogram.png"
plt.savefig(filename)
filename = measurements_directory + os.path.sep + "postprocessing_testbench_histogram.pdf"
fs = plt.rcParams.get('font.size')
plt.rcParams.update({'font.size': 20})
plt.savefig(filename, format='pdf', bbox_inches='tight', pad_inches=0.1)
plt.rcParams.update({'font.size': fs})
